# HW05 — Линейные модели и честный ML-эксперимент

Цель: сравнить бейзлайн-модель и логистическую регрессию на учебном датасете.

## 1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, RocCurveDisplay

import matplotlib.pyplot as plt
%matplotlib inline

## 2. Загрузка данных и первичный анализ

In [ ]:
df = pd.read_csv('S05-hw-dataset.csv')
df.head()

: 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['default'].value_counts(normalize=True)

В датасете около 3000 наблюдений и числовые признаки.
Целевой признак `default` умеренно несбалансирован (~40% положительного класса).

## 3. Подготовка признаков и таргета

In [ ]:
X = df.drop(columns=['default', 'client_id'])
y = df['default']
X.head()

## 4. Train/Test split и бейзлайн

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train, y_train)

y_pred_dummy = dummy.predict(X_test)
y_proba_dummy = dummy.predict_proba(X_test)[:, 1]

dummy_acc = accuracy_score(y_test, y_pred_dummy)
dummy_auc = roc_auc_score(y_test, y_proba_dummy)

dummy_acc, dummy_auc

Бейзлайн всегда предсказывает самый частый класс и служит точкой отсчёта.

## 5. Логистическая регрессия

In [ ]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

C_values = [0.01, 0.1, 1.0, 10.0]
results = []

for C in C_values:
    pipe.set_params(logreg__C=C)
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    results.append({'C': C, 'accuracy': acc, 'roc_auc': auc})

results_df = pd.DataFrame(results)
results_df

## 6. Лучшая модель и ROC-кривая

In [ ]:
best_row = results_df.sort_values('roc_auc', ascending=False).iloc[0]
best_C = best_row['C']

pipe.set_params(logreg__C=best_C)
pipe.fit(X_train, y_train)

y_proba_best = pipe.predict_proba(X_test)[:, 1]

RocCurveDisplay.from_predictions(y_test, y_proba_best)
plt.title('ROC-кривая для логистической регрессии')
plt.show()

## 7. Сравнение моделей и выводы

- Логистическая регрессия значительно превосходит бейзлайн по ROC-AUC.
- Accuracy также выше, что говорит о полезности признаков.
- Значение параметра C влияет на качество, оптимальное значение находится экспериментально.
- Логистическая регрессия выглядит разумной моделью для данной задачи как интерпретируемый и устойчивый метод.